In [ ]:
#packages
import torch 
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR
from accelerate import Accelerator
from tqdm.auto import tqdm
from sklearn.metrics import accuracy_score
import numpy as np
import accelerate as a 

#self made 
#import research as r
from model import SPHNet
from dataproccesing import process 
import dataproccesing as dp
from training import Train

In [13]:

dataPARAMS = {
    "file_path": "data/btc15m.csv",
    "date_column": "timestamp",
    "target_column": "close",
    "window_size": 256,
    "batch_size": 128,
    "val_size": 0.2,
    "test_size": 0.1,
    }
trainLoader, valLoader, testLoader, scaler = process(**dataPARAMS)

# infer number of features from the dataset (window samples shape = [N, seq_len, num_features])
num_features = trainLoader.dataset.X.shape[2]

modelPARAMS = {
    "num_features": num_features,
    "patch_size": 8,
    "embed_dim": 256,
    "vit_num_layers": 4,
    "transformer_num_layers": 4,
    "num_heads": 8,
    "ff_dim": 512,
    "dropout": 0.4,
    "output_dim": 1,
}
# make sure the class name matches what's defined in model.py
model = SPHNet(**modelPARAMS)

trainingPARAMS = {
    "valGAP": 1,
    "criterion": nn.MSELoss(),
    "epochs": 4,
    "model": model,
    "optimizer": torch.optim.Adam(model.parameters(), lr=1e-4),
    "trainLoader": trainLoader,
    "valLoader": valLoader,
    "testLoader": testLoader,
    "scheduler": StepLR(torch.optim.Adam(model.parameters(), lr=1e-4), step_size=10, gamma=0.1),
    "metrics": None,
}

# training the model

In [15]:
TrainedModel = Train(**trainingPARAMS)

Epoch 1 /4 | Train Loss: 0.0018: 100%|██████████| 556/556 [00:29<00:00, 19.11batch/s]


Epoch 1, Validation Loss: 0.000502


Epoch 2 /4 | Train Loss: 0.0012: 100%|██████████| 556/556 [00:29<00:00, 19.14batch/s]


Epoch 2, Validation Loss: 0.000418


Epoch 3 /4 | Train Loss: 0.0010: 100%|██████████| 556/556 [00:28<00:00, 19.27batch/s]


Epoch 3, Validation Loss: 0.000137


Epoch 4 /4 | Train Loss: 0.0009: 100%|██████████| 556/556 [00:28<00:00, 19.26batch/s]

Epoch 4, Validation Loss: 0.000488


Testing Loss: 0.0001647191140896604


In [16]:
torch.save(TrainedModel.state_dict(), 'models/sphnet_model_1re.pth')
print("Model saved successfully into 'models/sphnet_model_1re.pth'")

Model saved successfully into 'models/sphnet_model_1re.pth'


In [ ]:
torch.load(TrainedModel.state_dict(), 'models/sphnet_model_1re.pth')
print("Model loaded successfully from 'models/sphnet_model_1re.pth'")

# testing the model

based on the paper their are a few different tests that i can do

for price prediction:
    - r^2: a coefficient of dermination
    - MSE: avg squared distance of prediction
for direction prediction:
    - precision: TP / TP + FP
    - accuracy: (TP + FP) / (TP + FP + TN + FN)
    - recall: TP / TP + FN
TP, TN, FP, FN are defined as: (relative to A being up as a class)
    - TP: y == A and y-hat == A
    - TN: y != A and y-hat != A
    - FP: y == A but y-hat != A
    - FN: y != A but y-hat == A 